# Install

In [1]:
!pip install numba

from numba import cuda
device = cuda.get_current_device()
device.reset()

In [2]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [3]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U datasets
!pip install -qqq langchain==0.0.233 --progress-bar off

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 12.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 967.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.4 MB/s eta 0:00:00


# Import

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TextStreamer
import transformers
import torch
from torch.utils.data import DataLoader, Dataset
import torch
from transformers import AutoTokenizer, pipeline

# Load Quantized Model

In [45]:
model_id = "CleverShovel/vicuna-7b-v1.3-sharded-bf16" # sharded model by vilsonrodrigues
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model2 = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [46]:
from peft import prepare_model_for_kbit_training

model2.gradient_checkpointing_enable()
model2 = prepare_model_for_kbit_training(model2)

In [8]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [21]:
from peft import PeftModel, PeftConfig, LoraConfig
from transformers import AutoModelForCausalLM

config = LoraConfig.from_pretrained("FelixChao/vicuna-7b-instruct-ft-adapters-physics1.1")
# reload the base model (you might need pro subscription for this)

In [22]:
from peft import PeftModel

# load perf model with new adapters
model = PeftModel.from_pretrained(
    model,
    "FelixChao/vicuna-7b-instruct-ft-adapters-physics1.1",
    is_trainable=True,
)

# model.train()

In [ ]:
# from peft import LoraConfig, get_peft_model

# config = LoraConfig(
#     r=16,
#     lora_alpha=32,
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM",
# )


# model = get_peft_model(model, config)
# print_trainable_parameters(model)

trainable params: 13107200 || all params: 6685086720 || trainable%: 0.19606626733482374


# Prepare Data

In [12]:
from datasets import load_dataset

data = load_dataset("andersonbcdefg/physics")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [13]:
tokenizer.pad_token = tokenizer.eos_token

# prompt_template = "### Instruction: {prompt}\n### Response:"

train_dataset = data['train'].map(lambda x: {"input_text": x['message_1']  + "\n" + x['message_2']})

# Tokenize the datasets
train_encodings = tokenizer(train_dataset['input_text'], truncation=True, padding=True, max_length=256, return_tensors='pt')

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [14]:
class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = item["input_ids"].clone()
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


In [15]:
# Convert the encodings to PyTorch datasets
train_dataset = TextDataset(train_encodings)

In [16]:
streamer = TextStreamer(tokenizer)

# Example Before Fine Tuning

In [54]:
def generate(index):

  example_text = data['train'][index]['message_1']
  correct_answer = data['train'][index]['message_2']

  print("Question:")
  print(example_text)

  encoding = tokenizer(example_text, return_tensors="pt").to("cuda:0")
  output = model.generate(input_ids=encoding.input_ids, streamer = streamer, attention_mask=encoding.attention_mask, max_new_tokens=1000, do_sample=True, temperature=0.000001, eos_token_id=tokenizer.eos_token_id, top_k = 0)
  predict = tokenizer.decode(output[0], skip_special_tokens=True)
  predict = predict[len(example_text)+1:]
  # print("Answer:")
  # print(predict)

  # print("Best Answer:")
  # print(correct_answer)

  print()

In [53]:
generate(0) #base model

Question:
A heat engine operates between a hot temperature source of 400 K and a cold temperature source of 300 K. Calculate the Carnot efficiency of the heat engine.
<s> A heat engine operates between a hot temperature source of 400 K and a cold temperature source of 300 K. Calculate the Carnot efficiency of the heat engine.</s>



In [55]:
generate(0) #vicuna model finetune

Question:
A heat engine operates between a hot temperature source of 400 K and a cold temperature source of 300 K. Calculate the Carnot efficiency of the heat engine.
<s> A heat engine operates between a hot temperature source of 400 K and a cold temperature source of 300 K. Calculate the Carnot efficiency of the heat engine.
The Carnot efficiency is the maximum efficiency that a heat engine can achieve when operating between two temperatures. It is given by the formula:

Carnot efficiency = 1 - (T_cold / T_hot)

where T_cold is the temperature of the cold temperature source (300 K) and T_hot is the temperature of the hot temperature source (400 K).

Carnot efficiency = 1 - (300 K / 400 K)
Carnot efficiency = 1 - 0.75
Carnot efficiency = 0.25

So, the Carnot efficiency of the heat engine is 25%. This means that the heat engine can achieve a maximum efficiency of 25% when operating between the hot and cold temperature sources.</s>



In [ ]:
import random
random.seed(9)
rand_q = random.sample(list(range(len(data['train']))),10)

In [ ]:
def get_response_set(model):
   predictions = []

   for index in rand_q:
      example_text = data['train'][index]['message_1']
      encoding = tokenizer(example_text, return_tensors="pt").to("cuda:0")
      output = model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=2000, do_sample=True, temperature=0.000001, eos_token_id=tokenizer.eos_token_id, top_k = 0)
      predict = tokenizer.decode(output[0], skip_special_tokens=True)
      predict = predict[len(example_text)+1:]
      predictions.append(predict)

   return predictions


In [ ]:
predictions = get_response_set(model)

In [ ]:
predictions

['',
 'The balanced chemical equation for the neutralization reaction between sulfuric acid and sodium hydroxide is:\nH2SO4(aq) + NaOH(aq) → 2NaHSO4(aq)\nThis reaction involves the combination of a strong acid (sulfuric acid) and a strong base (sodium hydroxide) to produce a salt (sodium hydrogen sulfate) and water. The overall reaction is balanced by the equal number of hydrogen ions (H+) on both sides of the equation.',
 '',
 '\nThe activity of a catalyst can influence the reaction rate of the decomposition of hydrogen peroxide in several ways.\n\nFirstly, the catalyst can act as a surface for the reaction to occur on. This is known as surface catalysis. The catalyst can provide a low-energy surface for the reactants to come into contact with, which can increase the reaction rate.\n\nSecondly, the catalyst can act as a reagent in the reaction. This is known as catalytic reagent synthesis. The catalyst can be consumed in the reaction, which can increase the reaction rate.\n\nThirdly, 

In [ ]:
reference = data["train"]["message_2"]

In [ ]:
references = []
questions = []
for i in rand_q:
   references.append(reference[i])
   questions.append(data['train'][i]['message_1'])

references

['The solubility of sodium chloride (NaCl) in water at 25°C is approximately 36 grams per 100 mL of water. Therefore, the maximum amount of sodium chloride that can dissolve in 100 mL of water at 25°C is 36 grams.',
 'The balanced chemical equation for the neutralization reaction between sulfuric acid (H₂SO₄) and sodium hydroxide (NaOH) is:\n\n2 NaOH + H₂SO₄ → Na₂SO₄ + 2 H₂O\n\nIn this reaction, two moles of sodium hydroxide react with one mole of sulfuric acid to produce one mole of sodium sulfate and two moles of water.',
 'To calculate the standard enthalpy change (ΔH°) for the reaction, we can use the formula:\n\nΔH° = Σ [ΔHf°(products)] - Σ [ΔHf°(reactants)]\n\nwhere ΔHf° is the standard enthalpy of formation of each substance.\n\nFor the given reaction:\n\nΔH° = [1 * ΔHf°(Fe2O3)] - [2 * ΔHf°(Fe) + 3/2 * ΔHf°(O2)]\n\nSubstituting the given values:\n\nΔH° = [1 * (-826 kJ/mol)] - [2 * (-0.44 kJ/mol) + 3/2 * (0 kJ/mol)]\nΔH° = (-826 kJ/mol) - (-0.88 kJ/mol)\nΔH° = -825.12 kJ/mol\n\nN

In [ ]:
import pandas as pd

def compare_df(question,pred,ref) -> pd.DataFrame:
   info = {
      "quetsion" : question,
      "response" : pred,
      "answer" : ref,
   }

   return pd.DataFrame(info)


df = compare_df(questions,predictions,references)

In [ ]:
!pip install evaluate
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=4961f0af16c484cf8a225486b32cf5602da71bb25aed5c61ee7402d313973786
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
df

,quetsion,response,answer
0,What is the maximum amount of sodium chloride ...,,The solubility of sodium chloride (NaCl) in wa...
1,What is the balanced chemical equation for the...,The balanced chemical equation for the neutral...,The balanced chemical equation for the neutral...
2,Calculate the standard enthalpy change and ent...,,To calculate the standard enthalpy change (ΔH°...
3,"""How does the activity of a catalyst influence...",\nThe activity of a catalyst can influence the...,The activity of a catalyst influences the reac...
4,What is the most effective way to inhibit the ...,\nMolecular docking is a computational method ...,The most effective way to inhibit the interact...
5,How can we optimize the yield of polystyrene s...,\nPolystyrene is a synthetic polymer that is w...,To optimize the yield of polystyrene synthesis...
6,What is the standard enthalpy of formation of ...,,The given information is not sufficient to dir...
7,Calculate the standard electrode potential for...,,To calculate the standard electrode potential ...
8,"""How can the pharmacology of natural products ...",\nThe pharmacology of natural products is a co...,The pharmacology of natural products can be in...
9,What is the nuclear decay mode of a carbon-14 ...,The nuclear decay mode of a carbon-14 atom is ...,Carbon-14 undergoes beta-minus decay (β- decay...


In [ ]:
import evaluate

In [ ]:
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=predictions,references=references,use_stemmer=True)
results

{'rouge1': 0.2817929976488289,
 'rouge2': 0.12750407415897755,
 'rougeL': 0.18056557307575294,
 'rougeLsum': 0.24038532763071319}

# Training

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    # eval_dataset=val_dataset,
    args=transformers.TrainingArguments(
        num_train_epochs=1,
        per_device_train_batch_size=16,
        gradient_accumulation_steps=4,
        warmup_ratio=0.05,
        max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        lr_scheduler_type='cosine',
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

<ipython-input-11-424aeced090c>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


OutOfMemoryError: ignored

# Example After Fine Tuning

In [24]:
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              

In [ ]:
generate(3)

Question:
What volume of 0.2 M NaOH is required to completely neutralize 25 mL of 0.1 M HCl?
equation<s> What volume of 0.2 M NaOH is required to completely neutralize 25 mL of 0.1 M HCl?
To solve this problem, we can use the concept of moles and the balanced chemical equation for the reaction between NaOH and HCl:

NaOH + HCl → NaCl + H₂O

From the balanced equation, we can see that 1 mole of NaOH reacts with 1 mole of HCl.

First, we need to find the moles of HCl in the 25 mL of 0.1 M solution:

moles of HCl = Molarity × Volume
moles of HCl = 0.1 mol/L × 0.025 L (converting 25 mL to liters)
moles of HCl = 0.0025 mol

Since the reaction between NaOH and HCl is in a 1:1 ratio, we need to use 0.0025 mol of NaOH to neutralize the HCl.

Now, we can find the volume of NaOH required:

Volume of NaOH = Moles of NaOH / Molarity of NaOH
Volume of NaOH = 0.0025 mol / 0.0025 mol/L (since NaOH has the same molarity as HCl)
Volume of NaOH = 1 L

Since we need 1 L of NaOH to neutralize the HCl, we 

In [ ]:
prediction2 = get_response_set(model)

In [ ]:
prediction2

['To determine the maximum amount of sodium chloride (NaCl) that can dissolve in 100 mL of water at 25°C, we need to consider the solubility of NaCl in water.\n\nThe solubility of NaCl in water is approximately 35.2 g/100 mL at 25°C. This means that 35.2 g of NaCl can dissolve in 100 mL of water at 25°C.\n\nSo, the maximum amount of sodium chloride that can dissolve in 100 mL of water at 25°C is 35.2 g.\n\nPlease note that this is an approximation, and the actual amount of NaCl that can dissolve in 100 mL of water may vary slightly depending on the specific conditions of the experiment.',
 'The balanced chemical equation for the neutralization reaction between sulfuric acid (H₂SO₄) and sodium hydroxide (NaOH) is:\n\nH₂SO₄(aq) + 2NaOH(aq) → Na₂SO₄(aq) + 2H₂O(l)\n\nIn this reaction, one mole of sulfuric acid reacts with two moles of sodium hydroxide to produce one mole of sodium sulfate and two moles of water.',
 'To calculate the standard enthalpy change (ΔH°) for the reaction, we can u

In [ ]:
results2 = rouge.compute(predictions=prediction2,references=references,use_stemmer=True)

In [ ]:
results2

{'rouge1': 0.5521110495119378,
 'rouge2': 0.3381439439218584,
 'rougeL': 0.3784680506809217,
 'rougeLsum': 0.5111232522267442}

In [ ]:
bleu = evaluate.load('bleu')
results_bleu = bleu.compute(predictions=predictions, references=references)
results_bleu2 = bleu.compute(predictions=prediction2, references=references)

print(results_bleu)
print(results_bleu2)

{'bleu': 0.06389320763725576, 'precisions': [0.6257722859664607, 0.2839396628216504, 0.15611061552185548, 0.10224215246636771], 'brevity_penalty': 0.27687165053192225, 'length_ratio': 0.437789799072643, 'translation_length': 1133, 'reference_length': 2588}
{'bleu': 0.23996042784684243, 'precisions': [0.48559296329996965, 0.282628536659568, 0.18614586512053707, 0.1297826752372207], 'brevity_penalty': 1.0, 'length_ratio': 1.2739567233384854, 'translation_length': 3297, 'reference_length': 2588}


In [ ]:
toxicity = evaluate.load("toxicity")
results_tox = toxicity.compute(predictions=predictions, aggregation="ratio")
results_tox2 = toxicity.compute(predictions=prediction2, aggregation="ratio")
print(results_tox)
print(results_tox2)

{'toxicity_ratio': 0.0}
{'toxicity_ratio': 0.0}


In [ ]:
df.insert(loc = 2, column="vicuna7b_model_response", value=prediction2)

In [ ]:
df

,quetsion,response,vicuna7b_model_response,answer
0,What is the maximum amount of sodium chloride ...,,To determine the maximum amount of sodium chlo...,The solubility of sodium chloride (NaCl) in wa...
1,What is the balanced chemical equation for the...,The balanced chemical equation for the neutral...,The balanced chemical equation for the neutral...,The balanced chemical equation for the neutral...
2,Calculate the standard enthalpy change and ent...,,To calculate the standard enthalpy change (ΔH°...,To calculate the standard enthalpy change (ΔH°...
3,"""How does the activity of a catalyst influence...",\nThe activity of a catalyst can influence the...,The activity of a catalyst plays a crucial rol...,The activity of a catalyst influences the reac...
4,What is the most effective way to inhibit the ...,\nMolecular docking is a computational method ...,The most effective way to inhibit the interact...,The most effective way to inhibit the interact...
5,How can we optimize the yield of polystyrene s...,\nPolystyrene is a synthetic polymer that is w...,To optimize the yield of polystyrene synthesis...,To optimize the yield of polystyrene synthesis...
6,What is the standard enthalpy of formation of ...,,To find the standard enthalpy of formation of ...,The given information is not sufficient to dir...
7,Calculate the standard electrode potential for...,,To calculate the standard electrode potential ...,To calculate the standard electrode potential ...
8,"""How can the pharmacology of natural products ...",\nThe pharmacology of natural products is a co...,The pharmacology of natural products can be in...,The pharmacology of natural products can be in...
9,What is the nuclear decay mode of a carbon-14 ...,The nuclear decay mode of a carbon-14 atom is ...,Carbon-14 (C14) is a radioactive isotope of ca...,Carbon-14 undergoes beta-minus decay (β- decay...


In [ ]:
c1 = "rouge1"
c2 = "rouge2"
cL = "rougeL"
cLsum = "rougeLsum"
df_score = pd.DataFrame({
    c1 : [results[c1], results2[c1], results3[c1]],
    c2 : [results[c2], results2[c2], results3[c2]],
    cL : [results[cL], results2[cL], results3[cL]],
    cLsum : [results[cLsum], results2[cLsum], results3[cLsum]],
})
df_score.index = ["falcon7b" , "ESG_model", "vicuna7b"]
df_score

,rouge1,rouge2,rougeL,rougeLsum
falcon7b,0.466976,0.145024,0.273879,0.294289
ESG_model,0.525273,0.203351,0.325743,0.354960
vicuna7b,0.457632,0.168178,0.277941,0.301291


# Merge Adapters and Save Model to Hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.save_pretrained("vicuna-7b-instruct-ft-adapters-chemical1.4", push_to_hub=True) # save LORA adapters
model.push_to_hub("vicuna-7b-instruct-ft-adapters-chemical1.4")

adapter_model.bin:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/FelixChao/vicuna-7b-instruct-ft-adapters-chemical1.4/commit/45d8f401fec9fea7b1426aba86301b2347bddf37', commit_message='Upload model', commit_description='', oid='45d8f401fec9fea7b1426aba86301b2347bddf37', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model.print_trainable_parameters()

trainable params: 8,388,608 || all params: 6,746,804,224 || trainable%: 0.12433454005023165


In [ ]:
from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.llms import HuggingFacePipeline
from langchain.schema import BaseOutputParser

In [ ]:
model.generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1000,
  "pad_token_id": 2,
  "repetition_penalty": 1.7,
  "temperature": 1e-06,
  "top_k": 0,
  "transformers_version": "4.32.0.dev0",
  "use_cache": false
}

In [ ]:
generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task="text-generation",
    generation_config=generation_config,
)

llm = HuggingFacePipeline(pipeline=generation_pipeline)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead',

In [ ]:
prompt = "What volume of 0.2 M NaOH is required to completely neutralize 25 mL of 0.1 M HCl?"
res = llm(prompt)
print(res)


To solve this problem, we will use the concept called "Molarity and Concentration." Firstly, let's find out how many moles (moles) are present in both solutions:
